# Final Project 3

### Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
TASK 3: Add a Range Slider to Select Payload
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.

The github url and the screenshots are later required in the presentation slides.

In [57]:
# Import required libraries
import pandas as pd
import dash
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

In [58]:
#Jupyter Specific files
from jupyter_dash import JupyterDash

In [59]:

#Datasources and variables
Db_location= "C:/Users/brian/Downloads/spacex_launch_dash.csv"
# Read the data into pandas dataframe

spacex_df = pd.read_csv(Db_location)



#Global Variables
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
Sitegroup= spacex_df.groupby('Launch Site')

cld = spacex_df['class'].count()
#sld = spacex_df.groupby('Launch Site')['class'].sum()
sld = spacex_df['class'].sum()
spacex_df['perc'] = (cld) * 100


print (cld, sld)
print (spacex_df.columns)

56 24
Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category',
       'perc'],
      dtype='object')


#Datasources and variables
Db_location= "C:/Users/brian/Downloads/spacex_launch_dash.csv"
# Read the data into pandas dataframe

spacex_df = pd.read_csv(Db_location)



#Global Variables
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
Sitegroup= spacex_df.groupby('Launch Site')
cld= Sitegroup['class'].count()
sld =Sitegroup['class'].sum()
spacex_df['count']=cld
spacex_df['perc']= (cld/sld)* 100
print (spacex_df)

## TASK 1: Add a Launch Site Drop-down Input Component  (Task 3 below out of order)
We have four different launch sites and we would like to first see which one has the largest success count. Then, we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented dcc.Dropdown(id='site-dropdown',...) input with following attributes:
id attribute with value site-dropdown
options attribute is a list of dict-like option objects (with label and value attributes). You can set the label and value all to be the launch site names in the spacex_df and you need to include the default All option. e.g.,
  options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]
value attribute with default dropdown value to be ALL meaning all sites are selected
placeholder attribute to show a text description about this input area, such as Select a Launch Site here
searchable attribute to be True so we can enter keywords to search launch sites

## TASK 3: Add a Range Slider to Select Payload
Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we want to be able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented dcc.RangeSlider(id='payload-slider',...) input with the following attribute:

id to be payload-slider
min indicating the slider starting point, we set its value to be 0 (Kg)
max indicating the slider ending point to, we set its value to be 10000 (Kg)
step indicating the slider interval on the slider, we set its value to be 1000 (Kg)
value indicating the current selected range, we could set it to be min_payload and max_payload


In [60]:
# Create a dash application
app = dash.Dash(__name__)
#app=JupyterDash(__name__)

#app=Dash(__name__)

In [61]:
entered_site="ALL"

In [62]:
# Create an app layout   ***  Do NOT TOUCH ****
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                               dcc.Dropdown(id='site-dropdown',
                                               options=[{'label': 'All Sites', 'value': 'ALL'},
                                               {'label': 'LC-40', 'value': 'CCAFS LC-40'},
                                               {'label': 'SLC-40', 'value': 'CCAFS SLC-40'},
                                               {'label': 'LC-39A', 'value': 'KSC LC-39A'},
                                               {'label': 'SLC-4E', 'value': 'VAFB SLC-4E'}],
                                               value='ALL', 
                                               placeholder="Select a Launch Site here",
                                               searchable=True
                                               ),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                   #dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')
                             ),
                                html.Br(),

                                

                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):"),
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0',100: '100'},
                                value=[min_payload, max_payload]),
                                ),

                                # TASK 4: Add a scatter chart to show the correlation 
                                # between payload and launch success
                                html.Div(
                                 dcc.Graph(id='success-payload-scatter-chart'),
                                 )])

In [63]:
#####   DON' Touch ###############################
fig = px.pie(spacex_df, 
                values='class',  
                names='Launch Site', 
                title='Total Success Launches for'+" "+ entered_site)

fig.show()


### TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
The general idea of this callback function is to get the selected launch site from site-dropdown and render a pie chart visualizing launch success counts.

Dash callback function is a type of Python function which will be automatically called by Dash whenever receiving an input component updates, such as a click or dropdown selecting event.

In [64]:
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"), 
    Input(component_id= 'site-dropdown', component_property="value"))
def piechart(entered_site):
     #Filters and local variables
     cld = spacex_df['class'].count()
     sld = spacex_df['class'].sum()
     spacex_df['perc'] = (sld) 
     #Conditions
     if entered_site =="ALL":
          fig = px.pie(spacex_df, 
                values='class',  
                names='Launch Site', 
                title='Total Success Launches for'+" "+ entered_site)
          return fig
     else:
      Site= pd.DataFrame(Sitegroup.get_group(entered_site))
      Site['count']=Site['class'].sum()
      fig = px.pie(Site,
      values='count',
      names='class',
      title='Total Success Launches for'+" "+ entered_site)
      return fig

          
    

In [65]:
#piechart ('ALL')
#piechart('KSC LC-39A')
piechart('CCAFS SLC-40')

## TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., class column). As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

In addition, we want to color-label the Booster version on each scatter point so that we may observe mission outcomes with different boosters.

Now, let's add a call function including the following application logic:

Input to be [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")] Note that we have two input components, one to receive selected launch site and another to receive selected payload range
Output to be Output(component_id='success-payload-scatter-chart', component_property='figure')
A If-Else statement to check if ALL sites were selected or just a specific launch site was selected
If ALL sites are selected, render a scatter plot to display all values for variable Payload Mass (kg) and variable class.
In addition, the point color needs to be set to the booster version i.e., color="Booster Version Category"
If a specific launch site is selected, you need to filter the spacex_df first, and render a scatter chart to show
values Payload Mass (kg) and class for the selected site, and color-label the point using Boosster Version Category likewise.

Scatter Plot

In [66]:
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"), 
    Input(component_id= 'site-dropdown', component_property="value"), 
    Input(component_id= 'payload-slider',component_property= "value"))
def scatter_plt (entered_site, slider):
    filtered_df=Sitegroup.first()
    if entered_site == "ALL":
        fig = px.scatter (spacex_df,x= "Payload Mass (kg)", y= "class",color="Booster Version Category",)
        names='Booster Version Category' 
        title= 'title'
    else:
        Site= pd.DataFrame(Sitegroup.get_group(entered_site))
        #Site['count']=Site['class'].sum()
        #Site['Payload Mass (kg)']=slider['value']
        fig=px.scatter(Site,x= "Payload Mass (kg)", y= "class",)
        names= 'Booster Version Category'
        title= "Payload Slider"
    
    print(slider)
    return fig

In [67]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Dec/2022 21:31:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:25] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:25] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:31:26] "POST /_dash-update-component HTTP/1.1" 200 -


[0, 9600]


...

[0, 9600]


127.0.0.1 - - [21/Dec/2022 21:32:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:32:44] "POST /_dash-update-component HTTP/1.1" 200 -


[0, 9600]


127.0.0.1 - - [21/Dec/2022 21:35:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2022 21:35:07] "POST /_dash-update-component HTTP/1.1" 200 -


[0, 9600]
